In [1]:
import random
import numpy as np
import tensorflow as tf
import os

# Set seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [2]:
import pandas as pd
import numpy as np
import os

meta = pd.read_csv('Train_Test_Split.csv')
meta = meta[meta['label'].isin(['AD', 'Healthy'])]

train_meta = meta[meta['split'] == 'train']
test_meta = meta[meta['split'] == 'test']

def load_and_segment(subject_id, data_dir='Data_sampled_128HZ', segment_len=1024):
    file_path = os.path.join(data_dir, f"{subject_id}_data.npy")
    data = np.load(file_path)
    _, time_steps = data.shape
    num_segments = time_steps // segment_len
    if num_segments == 0:
        return np.empty((0, 19, segment_len))
    data = data[:, :num_segments * segment_len]
    segments = data.reshape(19, num_segments, segment_len).transpose(1, 0, 2)
    return segments

def process_data(meta_df, data_dir='Data_sampled_128HZ'):
    X = []
    y = []
    label_map = {'AD': 1, 'Healthy': 0}
    for _, row in meta_df.iterrows():
        segments = load_and_segment(row['subject_id'], data_dir)
        if segments.shape[0] == 0:
            continue
        X.append(segments)
        label = label_map[row['label']]
        one_hot = np.eye(2)[label]
        y.extend([one_hot] * segments.shape[0])
    X = np.concatenate(X, axis=0)
    y = np.array(y)
    return X, y
X_train, y_train = process_data(train_meta)
X_test, y_test = process_data(test_meta)
X_train = (X_train * 1e6) - np.mean(X_train * 1e6, axis=2, keepdims=True)
X_test = (X_test * 1e6) - np.mean(X_test * 1e6, axis=2, keepdims=True)

In [3]:
from tensorflow.keras.models import load_model, Model

# REPLACE YOUR FUNCTION DEFINITION WITH THIS
def load_and_trim_model(path, new_name, unfreeze_last_n_layers=0):
    """
    Loads a model, REMOVES its last layer, RENAMES it, and optionally
    unfreezes the last N layers for fine-tuning.
    """
    # Load the base model
    model = load_model(path)
    
    # Remove the original classification head
    model.pop() 

    # --- THE FIX ---
    # Assign the new, unique name using the public property
    model.name = new_name
    # ---------------

    if unfreeze_last_n_layers == 0:
        # Option 1: Freeze the entire model
        model.trainable = False
    else:
        # Option 2: Fine-tuning
        # Start by freezing everything
        model.trainable = False
        
        # Ensure N is not larger than the number of layers
        num_to_unfreeze = min(len(model.layers), unfreeze_last_n_layers)
        
        print(f"Unfreezing last {num_to_unfreeze} layers of {model.name}...")
        
        # Iterate over the last N layers and set them to trainable
        for layer in model.layers[-num_to_unfreeze:]:
            # We must keep Batch Norm layers frozen
            if not isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.trainable = True
                
    return model

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D, Concatenate, Lambda, Embedding
from tensorflow.keras.models import Model

def build_end_to_end_fusion_model(bilstm_model, cnn_time_model, cnn_freq_model, 
                                  common_dim=384, num_heads=10, ff_dim=256, dropout_rate=0.15):
    
    # === 1. DEFINE RAW DATA INPUTS ===
    # Input for the BiLSTM model (samples, 1024, 5)
    bilstm_raw_input = Input(shape=(1024, 5), name="bilstm_raw_input")
    
    # Input for the Time-domain CNN (samples, 19, 1024, 1)
    cnn_time_raw_input = Input(shape=(19, 1024, 1), name="cnn_time_raw_input")
    
    # Input for the Freq-domain CNN (samples, 19, 129, 1)
    # (Note: nperseg=256 gives 129 freq bins)
    cnn_freq_raw_input = Input(shape=(19, 129, 1), name="cnn_freq_raw_input")

    # === 2. PASS INPUTS THROUGH FEATURE EXTRACTORS ===
    feat_bilstm = bilstm_model(bilstm_raw_input)
    feat_cnn_time = cnn_time_model(cnn_time_raw_input)
    feat_cnn_freq = cnn_freq_model(cnn_freq_raw_input)

    # === 3. FUSION LOGIC (Your code) ===
    bilstm_proj = Dense(common_dim)(feat_bilstm)
    cnn_time_proj = Dense(common_dim)(feat_cnn_time)
    cnn_freq_proj = Dense(common_dim)(feat_cnn_freq)

    x = Lambda(lambda t: tf.stack(t, axis=1))([bilstm_proj, cnn_time_proj, cnn_freq_proj])

    for _ in range(2):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=common_dim)(x, x)
        x = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ffn_output = Dense(ff_dim, activation='relu')(x)
        ffn_output = Dense(common_dim)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(x + ffn_output)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(2, activation='softmax')(x)

    # === 4. CREATE THE FINAL MODEL ===
    model = Model(
        inputs=[bilstm_raw_input, cnn_time_raw_input, cnn_freq_raw_input], 
        outputs=out,
        name="End_To_End_Fusion_Model"
    )
    
    return model


In [5]:
# REPLACE YOUR MODEL LOADING/COMPILING CELL WITH THIS
# === 1. SET FINE-TUNING PARAMETERS ===
UNFREEZE_COUNT = 3 
LEARNING_RATE = 1e-5 

# === 2. LOAD MODELS using the NEW function ===
print("Loading and trimming models for fine-tuning...")

bilstm_model = load_and_trim_model(
    'Models/Final_Bilstm_model.keras', 
    new_name="bilstm_feature_extractor",  # <-- Pass unique name
    unfreeze_last_n_layers=UNFREEZE_COUNT
)
cnn_time_model = load_and_trim_model(
    'Models/Final_CNNSpatial_model.keras', 
    new_name="cnn_time_feature_extractor", # <-- Pass unique name
    unfreeze_last_n_layers=UNFREEZE_COUNT
)
cnn_freq_model = load_and_trim_model(
    'Models/Final_CNNSpectral_model.keras', 
    new_name="cnn_freq_feature_extractor", # <-- Pass unique name
    unfreeze_last_n_layers=UNFREEZE_COUNT
)

# === 3. BUILD THE END-TO-END MODEL ===
print("Building the end-to-end model...")
# (This part remains the same and will now work)
fusion_model = build_end_to_end_fusion_model(
    bilstm_model, cnn_time_model, cnn_freq_model,
    common_dim=384, num_heads=10, ff_dim=256, dropout_rate=0.15
)

# === 4. COMPILE with the new learning rate ===
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=LEARNING_RATE) 
fusion_model.compile(
    optimizer=optimizer, 
    loss='categorical_crossentropy', 
    metrics=['accuracy','AUC']
)

print("Model compiled successfully.")
print(fusion_model.summary())

Loading and trimming models for fine-tuning...
Unfreezing last 3 layers of bilstm_feature_extractor...
Unfreezing last 3 layers of cnn_time_feature_extractor...
Unfreezing last 3 layers of cnn_freq_feature_extractor...
Building the end-to-end model...

Model compiled successfully.


Model: "End_To_End_Fusion_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ bilstm_raw_input    │ (None, 1024, 5)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_time_raw_input  │ (None, 19, 1024,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_freq_raw_input  │ (None, 19, 129,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm_feature_ext… │ (None, 50)        │    328,970 │ bilstm_raw_input… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_time_feature_e… │ (None, 128)       │  8,407,936 │ cnn_time_raw_inp… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_freq_feature_e… │ (None, 128)       │  1,067,904 │ cnn_freq_raw_inp… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 384)       │     19,584 │ bilstm_feature_e… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 384)       │     49,536 │ cnn_time_feature… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 384)       │     49,536 │ cnn_freq_feature… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 3, 384)    │          0 │ dense[0][0],      │
│                     │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 3, 384)    │  5,910,144 │ lambda[0][0],     │
│ (MultiHeadAttentio… │                   │            │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 3, 384)    │          0 │ lambda[0][0],     │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 3, 384)    │        768 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3, 256)    │     98,560 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 3, 384)    │     98,688 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 3, 384)    │          0 │ layer_normalizat… │
│                     │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 3, 384)    │        768 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 3, 384)    │  5,910,144 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat

 Total params: 22,142,092 (84.47 MB)

 Trainable params: 21,781,772 (83.09 MB)

 Non-trainable params: 360,320 (1.37 MB)

None


In [6]:
# === 1. PREPARE TRAINING INPUTS ===
print("Preparing training data inputs...")

# BiLSTM Inputs (This is your existing code)
top_channels = [14, 2, 0, 18, 4]
X_train_selected = X_train[:, top_channels, :].transpose(0, 2, 1) # (samples, 1024, 5)
X_bilstm_input = X_train_selected 

# CNN-Time Inputs (This is your existing code)
X_train_cnn = X_train[..., np.newaxis] # shape: (N, 19, 1024, 1)
X_cnn_time_input = X_train_cnn

# CNN-Freq Inputs (This is your existing code)
from scipy.signal import welch
def compute_spectral_features(X, fs=128, nperseg=256):
    num_segments, num_channels, num_samples = X.shape
    psd_all = []
    for seg in X:
        seg_psd = []
        for ch in seg:
            freqs, psd = welch(ch, fs=fs, nperseg=nperseg)
            seg_psd.append(psd)
        psd_all.append(seg_psd)
    psd_all = np.array(psd_all)
    psd_all = np.log1p(psd_all)
    return psd_all, freqs

X_train_spec, freqs = compute_spectral_features(X_train)
X_train_spec = X_train_spec[..., np.newaxis] # shape: (N, 19, 129, 1)
X_cnn_freq_input = X_train_spec

# --- NEW: Group training inputs into a list ---
X_train_inputs = [X_bilstm_input, X_cnn_time_input, X_cnn_freq_input]
print(f"Grouped training inputs: 3 arrays with shapes {[x.shape for x in X_train_inputs]}")


# === 2. PREPARE TEST INPUTS (FOR FINAL EVALUATION) ===
# (This uses logic from your old data prep and evaluation sections)
print("Preparing test data inputs...")

# BiLSTM Test Inputs
X_test_selected = X_test[:, top_channels, :].transpose(0, 2, 1) # (samples, 1024, 5)
X_bilstm_input_test = X_test_selected

# CNNSpatial Test Inputs
X_test_cnn = X_test[..., np.newaxis] # shape: (N, 19, 1024, 1)
X_cnn_time_input_test = X_test_cnn

# CNNSpectral Test Inputs
X_test_spec, _ = compute_spectral_features(X_test)
X_cnn_freq_input_test = X_test_spec[..., np.newaxis] # shape: (N, 19, 129, 1)

# --- NEW: Group test inputs into a list ---
X_test_inputs = [X_bilstm_input_test, X_cnn_time_input_test, X_cnn_freq_input_test]
print(f"Grouped test inputs: 3 arrays with shapes {[x.shape for x in X_test_inputs]}")


# === 3. TRAIN THE MODEL ===
print("Starting end-to-end model training...")
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)
]

# --- MODIFIED: We DELETED the .predict() calls ---
# --- We now pass the RAW inputs list directly to .fit() ---
history = fusion_model.fit(
    X_train_inputs,  # <-- THE MAIN CHANGE IS HERE
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks = callbacks
)

Preparing training data inputs...
Grouped training inputs: 3 arrays with shapes [(5248, 1024, 5), (5248, 19, 1024, 1), (5248, 19, 129, 1)]
Preparing test data inputs...
Grouped test inputs: 3 arrays with shapes [(1159, 1024, 5), (1159, 19, 1024, 1), (1159, 19, 129, 1)]
Starting end-to-end model training...
Epoch 1/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - AUC: 0.9463 - accuracy: 0.8936 - loss: 0.2378 - val_AUC: 0.9912 - val_accuracy: 0.9562 - val_loss: 0.1187 - learning_rate: 1.0000e-05
Epoch 2/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - AUC: 0.9954 - accuracy: 0.9694 - loss: 0.0830 - val_AUC: 0.9910 - val_accuracy: 0.9552 - val_loss: 0.1167 - learning_rate: 1.0000e-05
Epoch 3/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - AUC: 0.9969 - accuracy: 0.9747 - loss: 0.0679 - val_AUC: 0.9913 - val_accuracy: 0.9619 - val_loss: 0.1131 - learning_rate: 1.0000e-05
Epoch 4/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - AUC: 0.9978 - accuracy: 0.9825 - loss: 0.0539 - val_AUC: 0.9906 - val_

In [7]:
# === 5. EVALUATE THE MODEL (SEGMENT-LEVEL) ===
print("\nEvaluating model on the test set (segment-level)...")
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

# Get probabilities
y_pred_prob = fusion_model.predict(X_test_inputs)
# Get class predictions
y_pred_classes = np.argmax(y_pred_prob, axis=1)
# Get true classes
y_true = np.argmax(y_test, axis=1)

print("Segment-Level ROC AUC:", roc_auc_score(y_true, y_pred_prob[:, 1]))
print("Segment-Level Average Precision:", average_precision_score(y_true, y_pred_prob[:, 1]))
print("\nSegment-Level Classification Report:")
print(classification_report(y_true, y_pred_classes))


Evaluating model on the test set (segment-level)...
37/37 ━━━━━━━━━━━━━━━━━━━━ 17s 447ms/step
Segment-Level ROC AUC: 0.9260543973160795
Segment-Level Average Precision: 0.9302775084143666

Segment-Level Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.77      0.84       535
           1       0.83      0.93      0.88       624

    accuracy                           0.86      1159
   macro avg       0.87      0.85      0.86      1159
weighted avg       0.87      0.86      0.86      1159



In [8]:
output_folder = "Models"
os.makedirs(output_folder,exist_ok=True)
model_path = os.path.join(output_folder,"Final_End_to_End_model.keras")
model.save(model_path)

NameError: name 'model' is not defined

In [9]:
from scipy.stats import mode

def patient_level_ensemble_2(fusion_model, meta_df, voting='soft'):
    """
    Evaluates the END-TO-END fusion model on a patient-by-patient basis.
    
    The 'fusion_model' argument is now the complete end-to-end model.
    """
    y_true = []
    y_pred = []
    y_prob = []

    for _, row in meta_df.iterrows():
        subject_id = row['subject_id']
        label_str = row['label']
        true_label = 1 if label_str == 'AD' else 0

        segments = load_and_segment(subject_id)
        if segments.shape[0] == 0:
              continue
        
        # --- Prepare all 3 raw inputs for this patient ---
        segments = (segments * 1e6) - np.mean(segments * 1e6, axis=2, keepdims=True)
        top_channels = [14, 2, 0, 18, 4]
        bilstm_input = segments[:, top_channels, :].transpose(0, 2, 1)
        cnn_time_input = segments[..., np.newaxis]
        spec_feats, _ = compute_spectral_features(segments)
        cnn_freq_input = spec_feats[..., np.newaxis]
        
        # Group inputs into a list
        model_inputs = [bilstm_input, cnn_time_input, cnn_freq_input]

        # --- Make ONE prediction with the end-to-end model ---
        preds = fusion_model.predict(model_inputs, verbose=0)
        # --- (No more separate .predict calls) ---

        if voting == 'soft':
            avg_prob = np.mean(preds, axis=0)
            y_pred.append(np.argmax(avg_prob))
            y_prob.append(avg_prob[1])
        elif voting == 'hard':
            pred_classes = np.argmax(preds, axis=1)
            voted_class = mode(pred_classes, keepdims=True).mode[0]
            y_pred.append(voted_class)
            # Use average probability for AUC, even in hard voting
            y_prob.append(np.mean(preds[:, 1])) 

        y_true.append(true_label)

    return np.array(y_true), np.array(y_pred), np.array(y_prob)

In [11]:
yt_hard, yp_hard, prob_hard = patient_level_ensemble_2(fusion_model, test_meta, voting='hard')

In [12]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

def evaluate_predictions(y_true, y_pred, y_prob, voting_type="Soft"):
    print(f"\n=== Patient-Level {voting_type} Voting Results ===")
    print(f"Patient-Level ROC AUC: {roc_auc_score(y_true, y_prob)}")
    print(f"Patient-Level Avg Precision: {average_precision_score(y_true, y_prob)}")
    print("\nPatient-Level Classification Report:")
    print(classification_report(y_true, y_pred))

# === 6. RUN PATIENT-LEVEL EVALUATION ===
print("\nRunning Patient-Level Evaluation (Soft Voting)...")
# Note: We pass the NEW end-to-end fusion_model
yt_soft, yp_soft, prob_soft = patient_level_ensemble_2(
    fusion_model, test_meta, voting='soft'
)
evaluate_predictions(yt_soft, yp_soft, prob_soft, voting_type="Soft")

print("\nRunning Patient-Level Evaluation (Hard Voting)...")
yt_hard, yp_hard, prob_hard = patient_level_ensemble_2(
    fusion_model, test_meta, voting='hard'
)
evaluate_predictions(yt_hard, yp_hard, prob_hard, voting_type="Hard")


Running Patient-Level Evaluation (Soft Voting)...

=== Patient-Level Soft Voting Results ===
Patient-Level ROC AUC: 1.0
Patient-Level Avg Precision: 1.0

Patient-Level Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.86      1.00      0.92         6

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.91        11
weighted avg       0.92      0.91      0.91        11


Running Patient-Level Evaluation (Hard Voting)...

=== Patient-Level Hard Voting Results ===
Patient-Level ROC AUC: 1.0
Patient-Level Avg Precision: 1.0

Patient-Level Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.86      1.00      0.92         6

    accuracy                           0.91        11
   macro avg       0.93      0.90      0.91        11
weighted avg     